See the google collaboratory for more details:
https://colab.research.google.com/drive/1mCbcWxvmARx1KTDbYHGVTNOa66Z4uCoJ?usp=sharing

In [1]:
# start meshcat server
server_args = []
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

In [2]:
import numpy as np
from functools import partial

from ipywidgets import FloatSlider, Layout, Dropdown, Textarea
from IPython.display import display, HTML, SVG

from pydrake.common.jupyter import process_ipywidget_events
from pydrake.systems.framework import BasicVector, VectorSystem

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)

from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.all import RigidTransform, RotationMatrix
from pydrake.geometry import Cylinder
from pydrake.multibody.tree import UnitInertia, SpatialInertia, JointIndex
from pydrake.multibody.plant import CoulombFriction
#from pydrake.multibody.inverse_kinematics import InverseKinematics
from PandaInverseKinematics import PandaInverseKinematics

In [3]:
def AddCylinder(plant, name, radius, length):

  mass = 1
  mu = 1
  color = [0.486, 0.305, 0.756,1]

  shape = Cylinder(radius, length)
  instance = plant.AddModelInstance(name)
  inertia = UnitInertia.SolidCylinder(shape.radius(), shape.length())

  body = plant.AddRigidBody(name, instance, SpatialInertia(mass=mass,p_PScm_E=np.array([0., 0., 0.]), G_SP_E=inertia))
    
  plant.RegisterCollisionGeometry(body, RigidTransform(), shape, name, CoulombFriction(mu, mu))

  plant.RegisterVisualGeometry(body, RigidTransform(), shape, name, color)

  return instance

def PoseWidget(publishing_system, root_context, callback):
  ''' create the slider widget that will callback with the desired position to the IK solver'''

  vars = ['x (m)','y (m)','z (m)','roll (rad)','pitch (rad)','yaw (rad)']
  desired_pose = np.array([0.3,0.5,0.3,0,np.pi/2,np.pi/2]) # initial slider (and hand) pose

  publishing_context = publishing_system.GetMyContextFromRoot(root_context) # Returns the mutable subsystem context for this system, given a root context.
  
  callback(root_context, desired_pose)
  publishing_system.Publish(publishing_context)

  def slider_callback(change, index):
    desired_pose[index] = change.new
    callback(root_context, desired_pose)
    publishing_system.Publish(publishing_context)

  slider_widgets = []

  
  max_vals = [0.6, 0.6, 0.7, np.pi, np.pi, np.pi]
  min_vals = [-0.6, -0.6, 0, -np.pi, -np.pi, -np.pi]

  for i in range(len(vars)):
    slider = FloatSlider(max=max_vals[i],
                             min=min_vals[i],
                             value=desired_pose[i],
                             step=0.01,
                             continuous_update=True,
                             description=vars[i],
                             layout=Layout(width='90%'))
    slider.observe(partial(slider_callback, index = i), names='value')
    display(slider)
    slider_widgets.append(slider)

  return slider_widgets

q_previous = None

In [4]:
def AddCollisionConstraints(ik, min_distance, arm_geometries, collidable_geometries):
  for i in range(len(arm_geometries)):
    for j in range(len(collidable_geometries)):
      pair = (arm_geometries[i], collidable_geometries[j])
      ik.AddDistanceConstraint(pair, min_distance, 1000) # we dont care about the upper bound so set it to a large number


def main():
    
  position_tolerance = 0.01

  panda_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm.urdf")
  hand_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/hand.urdf")

  builder = DiagramBuilder()
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
  panda_model = Parser(plant, scene_graph).AddModelFromFile(panda_file, "panda")
  hand_model = Parser(plant, scene_graph).AddModelFromFile(hand_file)
  plant.WeldFrames(plant.world_frame(),plant.GetFrameByName("panda_link0"))
  plant.WeldFrames(plant.GetFrameByName("panda_link8"),plant.GetFrameByName("panda_hand"))

  c1 = AddCylinder(plant, "c1", 0.05, 0.5)
  plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, 0.15, 0.25])) # add a model to test collisions

  plant.Finalize()

  meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

  diagram = builder.Build()
  context = diagram.CreateDefaultContext()
  plant_context = plant.GetMyContextFromRoot(context)
  

  q_nominal = plant.GetPositions(plant_context, panda_model)
  q_previous = None
  end_effector_frame = plant.GetFrameByName('panda_hand')
  world_frame = plant.world_frame()
  console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

  arm_geometries = []
  bodies = plant.GetBodyIndices(plant.GetModelInstanceByName("panda"))
  for i in bodies:
    b = plant.get_body(i)
    if (b.name() == "panda_link0"): # this is because the distance constraint is not supported between two static objects in the scene
        continue
    arm_geometries+=plant.GetCollisionGeometriesForBody(b)
        
  collidable_names = ["c1"]
  to_avoid_geometries = []
  for name in collidable_names:
    bodies = plant.GetBodyIndices(plant.GetModelInstanceByName(name))
    for i in bodies:
      to_avoid_geometries+=plant.GetCollisionGeometriesForBody(plant.get_body(i))

  def IK_callback(context, desired_pose):
    global q_previous
    ik =PandaInverseKinematics(plant, plant_context, panda_model)

    # position constraint
    p_tol = np.ones(3)*0.01
    ik.AddPositionConstraint(desired_pose[0:3]-p_tol, desired_pose[0:3]+ p_tol)
    #ik.AddPointToPointDistanceConstraint(end_effector_frame, np.array([0,0,0]), world_frame, desired_pose[0:3], 0, position_tolerance) # also can use AddPositionConstraint
    
    # add collision constraint
    # ik.AddMinimumDistanceConstraint(0.01, 0.1) # this should work but doesn't for this franka-panda model (it works for the IIWA)
    #AddCollisionConstraints(ik, 0.01,arm_geometries, to_avoid_geometries)

    # add orientation constraint
    rot = RigidTransform(RollPitchYaw(desired_pose[3:]), desired_pose[0:3]).rotation()
    #ik.AddOrientationConstraint(end_effector_frame, RotationMatrix(), world_frame, rot, 0)
    ik.AddOrientationConstraint(rot, 0.01)

    # set inital guess and solve
    prog = ik.get_prog()
    q = ik.get_q()
    # prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q) # optional
    if (q_previous is None):
      prog.SetInitialGuess(q, q_nominal)
    else:
      prog.SetInitialGuess(q, q_previous)

    result = Solve(prog)
    q_previous = result.GetSolution()
    console.value += str(result.is_success()) + "\n"
    plant.SetPositions(plant_context, panda_model, q_previous)
    
  meshcat.load()
  PoseWidget(meshcat, context, IK_callback)
  display(console)

In [24]:
main()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.
[-0.79450873 -1.11376317  1.68363313 -2.30216997  2.89534103  1.68935824
  1.19539019]


[-0.79450873 -1.11376317  1.68363313 -2.30216997  2.89534103  1.68935824
  1.19539019]
[-0.79450873 -1.11376317  1.68363313 -2.30216997  2.89534103  1.68935824
  1.19539019]
[-0.7945229  -1.10906721  1.66774174 -2.29609801  2.8973      1.68818838
  1.19545196]
[-0.79321912 -1.10757668  1.64681165 -2.29028464  2.87951034  1.69399558
  1.20756525]
[-0.79138842 -1.10215975  1.62796208 -2.28374445  2.86406441  1.69820436
  1.22341947]
[-0.7896058  -1.10121036  1.60771441 -2.2765033   2.84593854  1.70307344
  1.23548609]
[-0.78700535 -1.09485544  1.56959503 -2.26184836  2.81324619  1.71170935
  1.26394538]
[-0.78586091 -1.08992578  1.55156293 -2.25440562  2.79873235  1.71536963
  1.2796877 ]
[-0.78568319 -1.08852144  1.53207232 -2.24705192  2.78156741  1.72058935
  1.29308944]
[-0.78292484 -1.0844043   1.51450822 -2.23854468  2.76719363  1.72408722
  1.30831035]
[-0.7789953  -1.08133581  1.49667848 -2.22959921  2.75145249  1.72841075
  1.32381949]
[-0.77260172 -1.08039317  1.47731117 -2.223

[-1.08160807 -1.41927966  1.64128234 -1.41942685  2.87413414  3.4977791
  1.69112148]
[-1.05581181 -1.42521595  1.62974035 -1.4627753   2.87568428  3.42650723
  1.69485059]
[-1.00842447 -1.45718667  1.59588787 -1.54357509  2.88805115  3.28759659
  1.70302476]
[-0.97480979 -1.46554954  1.58069582 -1.60697314  2.8973      3.16973571
  1.70838032]
[-0.97009033 -1.47257017  1.57573474 -1.61642609  2.8973      3.14911628
  1.71336944]
[-0.92623802 -1.51709775  1.54441442 -1.70061833  2.8973      3.0064468
  1.76423801]
[-0.91617289 -1.52287873  1.53995345 -1.71332488  2.8973      2.97067074
  1.76381854]
[-0.90605895 -1.53757729  1.53158454 -1.73137839  2.8973      2.93810937
  1.77320228]
[-0.84897889 -1.61213444  1.49511499 -1.82579174  2.8973      2.80991946
  1.84415128]
[-0.84580602 -1.63701907  1.48339479 -1.83060563  2.8973      2.76606618
  1.86985975]
[-0.84380569 -1.66803844  1.47023474 -1.84455467  2.8973      2.73219025
  1.89984464]
[-0.84569026 -1.7628      1.43080796 -1.87513

[-1.28681613 -0.78405071  1.90402642 -1.83497959  2.00116869  0.96493233
  1.01546251]
[-1.29555015 -0.78678179  1.90804293 -1.82768238  1.99474029  0.94623466
  1.0091743 ]
[-1.32966699 -0.80423733  1.91775874 -1.78860286  1.95246141  0.86840464
  0.991959  ]
[-1.34225408 -0.81031832  1.92230666 -1.77447289  1.93976201  0.84053046
  0.98315351]
[-1.37148517 -0.8271649   1.93081741 -1.73669452  1.90426592  0.77383883
  0.96488234]
[-1.38349025 -0.8343634   1.93482303 -1.72051306  1.89161866  0.74725888
  0.95452417]
[-1.42098523 -0.85896092  1.94695805 -1.66418986  1.84989444  0.66578294
  0.92784679]
[-1.43255375 -0.86833264  1.95007071 -1.64342453  1.83679281  0.64142796
  0.91712953]
[-1.44506098 -0.87758154  1.95437606 -1.6218701   1.82445878  0.61608063
  0.90762654]
[-1.45321084 -0.88365905  1.9572294  -1.60766868  1.81677389  0.59954345
  0.90028449]
[-1.46557069 -0.89362293  1.960877   -1.58447642  1.80333547  0.57408233
  0.89129658]
[-1.48218375 -0.90647253  1.96654021 -1.553

[-0.59014176 -0.56881106  1.78323972 -2.15195102  2.79729984  2.29868822
  0.79683372]
[-0.56314666 -0.56720876  1.77668129 -2.15815098  2.81577878  2.31147553
  0.78320288]
[-0.53412142 -0.56425968  1.76763753 -2.16378772  2.83318795  2.32491544
  0.76986231]
[-0.50618493 -0.56160692  1.75967524 -2.16886727  2.8507476   2.3380472
  0.75832624]
[-0.45017095 -0.55918432  1.74279999 -2.17646059  2.88707723  2.3659588
  0.73446132]
[-0.45017095 -0.55918432  1.74279999 -2.17646059  2.88707723  2.3659588
  0.73446132]
[-0.45017095 -0.55918432  1.74279999 -2.17646059  2.88707723  2.3659588
  0.73446132]
[-0.47942943 -0.56059184  1.73560959 -2.18961958  2.8658077   2.34309325
  0.75114161]
[-0.49342652 -0.55676032  1.72796415 -2.18909096  2.84528164  2.3271085
  0.75316716]
[-0.56115679 -0.52242346  1.71498287 -2.19975559  2.75303954  2.26812519
  0.79558292]
[-0.62254197 -0.52809563  1.73473013 -2.17735654  2.70942438  2.25728492
  0.81163325]
[-0.64627796 -0.522636    1.73710541 -2.16455202

...

[ 0.02200829  1.7628      1.62369207 -2.20103444  2.8973      1.81168704
 -1.71165315]
[ 0.02438424  1.7628      1.62561534 -2.20237832  2.8973      1.79858091
 -1.71503603]
[ 0.07646035  1.7628      1.62124462 -2.23258383  2.82068578  1.31436749
 -1.79529143]
[ 0.09079535  1.7628      1.62077893 -2.23019569  2.81704855  1.25730532
 -1.80271308]
[ 0.09691519  1.76111942  1.62118915 -2.22904821  2.81044532  1.23292354
 -1.80575056]
[ 0.1212189   1.75630475  1.61541241 -2.22224899  2.79058054  1.13552912
 -1.82521384]
[ 0.13046013  1.75596432  1.61750805 -2.22012804  2.78196904  1.09965602
 -1.82762732]
[ 0.15821452  1.74737696  1.60922288 -2.2080674   2.75979224  0.99358529
 -1.85195409]
[ 0.2116437   1.73563173  1.60457384 -2.16797804  2.72615008  0.85027178
 -1.87338618]
[ 0.23148441  1.72706282  1.60127594 -2.15080206  2.71624566  0.79793851
 -1.8887104 ]
[ 0.28242049  1.71131259  1.59590646 -2.10201057  2.68995375  0.67134803
 -1.91322022]
[ 0.31432849  1.69565143  1.58901813 -2.066

[ 1.8580086   0.53312023 -0.61408275 -2.09923407  2.8973      3.7525
 -1.60442775]
[ 1.91632495  0.54037349 -0.6722904  -2.10508555  2.8973      3.7525
 -1.59721265]
[ 1.97326999  0.54890657 -0.72948778 -2.11148021  2.8973      3.7525
 -1.59058296]
[ 2.01040167  0.55527912 -0.76699011 -2.11604071  2.8973      3.7525
 -1.58650389]
[ 2.04682472  0.5621818  -0.80394767 -2.12083196  2.8973      3.7525
 -1.5827081 ]
[ 2.13462141  0.58164347 -0.89382036 -2.13379249  2.8973      3.7525
 -1.57453239]
[ 2.15160412  0.58589672 -0.91134974 -2.13654776  2.8973      3.7525
 -1.5731347 ]
[ 2.8973      1.05639909  2.79415305 -2.96798474  2.8973      3.7525
  2.82634738]
[ 2.8973      1.1011371   2.79232757 -2.98453488  2.8973      3.7525
  2.83142903]
[ 2.8973      1.13041909  2.79086209 -2.99496144  2.8973      3.7525
  2.8346539 ]
[ 2.8973      1.15932569  2.78925869 -3.00496771  2.8973      3.7525
  2.83780891]
[ 2.8973      1.17365197  2.78841827 -3.00982808  2.8973      3.7525
  2.83937304]
[ 2.

[ 2.02197333  0.63185479 -0.58156528 -1.920173   -1.86786938  1.0553279
  2.44870481]
[ 2.00084299  0.57780993 -0.54361226 -1.92697405 -1.80918071  1.14671932
  2.4356917 ]
[ 1.98455209  0.54007212 -0.51875138 -1.93212668 -1.76509714  1.21472633
  2.42353447]
[ 1.968981    0.50615443 -0.4977697  -1.93768512 -1.72259964  1.27737486
  2.40963299]
[ 1.96300795  0.49345167 -0.49050286 -1.94013692 -1.70738712  1.30230161
  2.40355969]
[ 1.94964527  0.46660348 -0.47509352 -1.94533372 -1.66962432  1.35316581
  2.3904032 ]
[ 1.92348553  0.41787126 -0.44901995 -1.95705478 -1.59704073  1.44748892
  2.35936085]
[ 1.91849963  0.40930013 -0.44488515 -1.95922803 -1.58520733  1.46694605
  2.35428469]
[ 1.91508804  0.40328978 -0.44209595 -1.96093657 -1.5764762   1.47999955
  2.34986592]
[ 1.90211095  0.38113731 -0.43166988 -1.96724019 -1.53793959  1.52503038
  2.33464884]
[ 1.9006928   0.37859037 -0.43070439 -1.96828638 -1.53516745  1.53096252
  2.33120907]
[ 1.88748002  0.35691181 -0.42122855 -1.9752

[ 1.60527158  0.41647686 -0.08259658 -1.9453898  -1.79991671  1.22486916
  2.35319717]
[ 1.59079337  0.41072387 -0.06811849 -1.94471995 -1.75447296  1.27345463
  2.34971225]
[ 1.58082565  0.40110116 -0.05886109 -1.94230435 -1.72352363  1.30600219
  2.34930806]
[ 1.5788556   0.39873545 -0.05723964 -1.94265304 -1.71867981  1.31340006
  2.34787001]
[ 1.56832021  0.38819849 -0.04832434 -1.94131007 -1.68682146  1.34852828
  2.34020675]
[ 1.54245775  0.36396401 -0.02884105 -1.93911527 -1.61170348  1.43280145
  2.31425196]
[ 1.53188827  0.35419478 -0.02194316 -1.93922473 -1.58256221  1.46805363
  2.30411552]
[ 1.5278562   0.35033899 -0.01951685 -1.93971746 -1.57233292  1.48200899
  2.29890084]
[ 1.51538012  0.34918972 -0.0269491  -1.9397602  -1.57580398  1.49886026
  2.29426331]
[ 1.49299866  0.35007153 -0.04418148 -1.93833944 -1.60091154  1.5303637
  2.29578334]
[ 1.44489725  0.3448407  -0.07651976 -1.93004406 -1.63867511  1.60238592
  2.29123178]
[ 1.42182078  0.35562703 -0.09339926 -1.9303

[ 2.44140073 -1.14555556 -0.07958139 -2.54957385 -1.64711458  1.73727621
  1.35903677]
[ 2.50809751 -1.19779127 -0.14491416 -2.56489558 -1.67389356  1.86342686
  1.30712365]
[ 2.5128608  -1.20035722 -0.14864436 -2.56471355 -1.67090511  1.868919
  1.31067269]
[ 2.54652455 -1.22468275 -0.1819216  -2.56987428 -1.68119764  1.93031149
  1.28512042]
[ 2.58046561 -1.24784961 -0.21492736 -2.57346825 -1.68795012  1.98941568
  1.26091924]
[ 2.59012162 -1.25384816 -0.22367203 -2.57389219 -1.68710242  2.00406407
  1.25887358]
[ 2.59909382 -1.25974405 -0.23229947 -2.57459033 -1.68841976  2.01923739
  1.25308206]
[ 2.65762022 -1.29751312 -0.28978563 -2.57831983 -1.70156934  2.12115133
  1.1926718 ]
[ 2.66827492 -1.30330366 -0.29794577 -2.57788881 -1.69452834  2.13314291
  1.19427125]
[ 2.67771554 -1.30888465 -0.30651314 -2.57796581 -1.69357477  2.14753854
  1.18679759]
